In [1]:

"""
Complete PINN-KAN Pipeline for Burgers Equation
Includes: PINN-KAN, Vanilla MLP, Vanilla PINN
With comprehensive visualization and metrics
"""

import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from typing import Dict, List, Tuple, Optional
import os
import pickle
from scipy.interpolate import griddata

# Set random seeds for reproducibility
torch.manual_seed(42)
np.random.seed(42)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cpu


In [2]:
class RBFEdge(nn.Module):
    """Gaussian Radial Basis Function layer."""
    def __init__(self, input_dim: int, num_rbfs: int):
        super().__init__()
        self.centers = nn.Parameter(torch.randn(num_rbfs, input_dim))
        self.sigmas = nn.Parameter(torch.ones(num_rbfs, input_dim))
    
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        # x: (batch, input_dim)
        # centers: (num_rbfs, input_dim)
        expanded = (x.unsqueeze(1) - self.centers) ** 2  # (batch, num_rbfs, input_dim)
        scaled = expanded / (2 * (self.sigmas ** 2))
        return torch.exp(-scaled.sum(dim=-1))  # (batch, num_rbfs)


class KANLayer(nn.Module):
    """KAN layer with RBF edge."""
    def __init__(self, input_dim: int, num_rbfs: int, output_dim: int):
        super().__init__()
        self.rbf_edge = RBFEdge(input_dim, num_rbfs)
        self.linear = nn.Linear(num_rbfs, output_dim)
    
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        phi = self.rbf_edge(x)
        return self.linear(phi)

In [3]:
class PINN_KAN(nn.Module):
    """Physics-Informed KAN for Burgers equation."""
    def __init__(self, input_dim: int = 2, num_rbfs_list: List[int] = [20, 30, 15], 
                 out_dim: int = 1):
        super().__init__()
        layers = []
        
        # First layer
        layers.append(KANLayer(input_dim, num_rbfs_list[0], num_rbfs_list[1]))
        layers.append(nn.ReLU())
        
        # Hidden layers
        for i in range(1, len(num_rbfs_list) - 1):
            layers.append(KANLayer(num_rbfs_list[i], num_rbfs_list[i], num_rbfs_list[i+1]))
            layers.append(nn.ReLU())
        
        # Output layer
        layers.append(nn.Linear(num_rbfs_list[-1], out_dim))
        
        self.model = nn.Sequential(*layers)
    
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return self.model(x)


class VanillaMLP(nn.Module):
    """Pure data-driven MLP (no physics loss)."""
    def __init__(self, input_dim: int = 2, hidden_dims: List[int] = [64, 64, 32], 
                 out_dim: int = 1):
        super().__init__()
        layers = []
        
        # Input layer
        layers.append(nn.Linear(input_dim, hidden_dims[0]))
        layers.append(nn.Tanh())
        
        # Hidden layers
        for i in range(len(hidden_dims) - 1):
            layers.append(nn.Linear(hidden_dims[i], hidden_dims[i+1]))
            layers.append(nn.Tanh())
        
        # Output layer
        layers.append(nn.Linear(hidden_dims[-1], out_dim))
        
        self.model = nn.Sequential(*layers)
    
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return self.model(x)


class VanillaPINN(nn.Module):
    """Vanilla PINN (MLP + Physics loss)."""
    def __init__(self, input_dim: int = 2, hidden_dims: List[int] = [64, 64, 32], 
                 out_dim: int = 1):
        super().__init__()
        layers = []
        
        # Input layer
        layers.append(nn.Linear(input_dim, hidden_dims[0]))
        layers.append(nn.Tanh())
        
        # Hidden layers
        for i in range(len(hidden_dims) - 1):
            layers.append(nn.Linear(hidden_dims[i], hidden_dims[i+1]))
            layers.append(nn.Tanh())
        
        # Output layer
        layers.append(nn.Linear(hidden_dims[-1], out_dim))
        
        self.model = nn.Sequential(*layers)
    
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return self.model(x)

In [4]:
def burgers_pde_residual(model: nn.Module, x: torch.Tensor, t: torch.Tensor, 
                         nu: float) -> torch.Tensor:
    """
    Compute Burgers equation residual: u_t + u*u_x - nu*u_xx = 0
    
    Args:
        model: Neural network model
        x: Spatial coordinate
        t: Temporal coordinate
        nu: Viscosity coefficient
    
    Returns:
        PDE residual tensor
    """
    X = torch.stack([x, t], dim=1).requires_grad_()
    u = model(X)
    
    # First derivatives
    grads = torch.autograd.grad(
        outputs=u, inputs=X, grad_outputs=torch.ones_like(u),
        create_graph=True, retain_graph=True, only_inputs=True
    )[0]
    u_x = grads[:, 0:1]
    u_t = grads[:, 1:2]
    
    # Second derivative
    u_xx = torch.autograd.grad(
        outputs=u_x, inputs=X, grad_outputs=torch.ones_like(u_x),
        create_graph=True, retain_graph=True, only_inputs=True
    )[0][:, 0:1]
    
    # Burgers equation residual
    residual = u_t + u * u_x - nu * u_xx
    
    return residual

In [5]:
# ==================== LOSS FUNCTIONS ====================

def data_loss(model: nn.Module, X: torch.Tensor, y: torch.Tensor) -> torch.Tensor:
    """Pure data loss (for Vanilla MLP)."""
    pred = model(X)
    return nn.MSELoss()(pred, y)


def pinn_loss(model: nn.Module, X_data: torch.Tensor, y_data: torch.Tensor,
              X_coll: torch.Tensor, nu: float, alpha: float = 1.0, 
              beta: float = 1.0) -> Tuple[torch.Tensor, torch.Tensor, torch.Tensor]:
    """
    Combined PINN loss: data + physics
    
    Args:
        model: Neural network
        X_data: Data points (x, t)
        y_data: True solution values
        X_coll: Collocation points
        nu: Viscosity
        alpha: Weight for data loss
        beta: Weight for physics loss
    
    Returns:
        Tuple of (total_loss, data_loss, physics_loss)
    """
    # Data loss
    pred = model(X_data)
    loss_data = nn.MSELoss()(pred, y_data)
    
    # Physics loss
    x_col = X_coll[:, 0]
    t_col = X_coll[:, 1]
    physics_residual = burgers_pde_residual(model, x_col, t_col, nu)
    loss_physics = torch.mean(physics_residual**2)
    
    # Total loss
    total = alpha * loss_data + beta * loss_physics
    
    return total, loss_data, loss_physics

In [6]:
# ==================== TRAINING ====================

def train_vanilla_mlp(model: nn.Module, X_data: torch.Tensor, y_data: torch.Tensor,
                      epochs: int = 2000, lr: float = 1e-3, 
                      print_every: int = 200) -> Dict[str, List[float]]:
    """Train vanilla MLP (data-driven only)."""
    optimizer = optim.Adam(model.parameters(), lr=lr)
    loss_history = []
    
    for epoch in range(epochs):
        model.train()
        optimizer.zero_grad()
        
        loss = data_loss(model, X_data, y_data)
        loss.backward()
        optimizer.step()
        
        loss_history.append(loss.item())
        
        if (epoch + 1) % print_every == 0 or epoch == 0:
            print(f"Epoch {epoch+1}/{epochs} | Loss: {loss.item():.6e}")
    
    return {'loss': loss_history}


def train_pinn(model: nn.Module, X_data: torch.Tensor, y_data: torch.Tensor,
               X_coll: torch.Tensor, nu: float, epochs: int = 2000, lr: float = 1e-3,
               alpha: float = 1.0, beta: float = 1.0, 
               print_every: int = 200) -> Dict[str, List[float]]:
    """Train PINN model (data + physics)."""
    optimizer = optim.Adam(model.parameters(), lr=lr)
    
    loss_history = []
    data_loss_history = []
    physics_loss_history = []
    
    for epoch in range(epochs):
        model.train()
        optimizer.zero_grad()
        
        total_loss, d_loss, p_loss = pinn_loss(
            model, X_data, y_data, X_coll, nu, alpha=alpha, beta=beta
        )
        
        total_loss.backward()
        optimizer.step()
        
        loss_history.append(total_loss.item())
        data_loss_history.append(d_loss.item())
        physics_loss_history.append(p_loss.item())
        
        if (epoch + 1) % print_every == 0 or epoch == 0:
            print(f"Epoch {epoch+1}/{epochs} | Total: {total_loss.item():.6e} | "
                  f"Data: {d_loss.item():.6e} | Physics: {p_loss.item():.6e}")
    
    return {
        'loss': loss_history,
        'data_loss': data_loss_history,
        'physics_loss': physics_loss_history
    }

In [7]:
# ==================== EVALUATION ====================

def compute_metrics(model: nn.Module, X: torch.Tensor, y: torch.Tensor,
                   X_coll: torch.Tensor, nu: float) -> Dict[str, float]:
    """Compute comprehensive evaluation metrics."""
    model.eval()
    with torch.no_grad():
        u_pred = model(X)
        
        # Prediction metrics
        mse = torch.mean((u_pred - y)**2)
        rmse = torch.sqrt(mse)
        mae = torch.mean(torch.abs(u_pred - y))
        rel_l2_error = torch.norm(u_pred - y) / torch.norm(y)
    
    # Physics residual metrics (requires gradients)
    X_coll_eval = X_coll.clone().detach().requires_grad_(True)
    x_col = X_coll_eval[:, 0]
    t_col = X_coll_eval[:, 1]
    
    residual = burgers_pde_residual(model, x_col, t_col, nu)
    
    residual_l2_norm = torch.norm(residual, p=2)
    residual_l2_normalized = residual_l2_norm / np.sqrt(len(residual))
    max_residual = torch.max(torch.abs(residual))
    mean_residual = torch.mean(torch.abs(residual))
    
    metrics = {
        'RMSE': rmse.item(),
        'MAE': mae.item(),
        'Relative_L2_Error': rel_l2_error.item(),
        'Residual_L2_Norm': residual_l2_normalized.item(),
        'Max_Residual': max_residual.item(),
        'Mean_Residual': mean_residual.item()
    }
    
    return metrics


In [8]:
# ==================== VISUALIZATION ====================

def plot_prediction_vs_actual(model: nn.Module, X: torch.Tensor, y: torch.Tensor,
                              title: str = "Predictions vs Actual"):
    """Scatter plot of predictions vs actual values."""
    model.eval()
    with torch.no_grad():
        preds = model(X).cpu().numpy()
        actual = y.cpu().numpy()
    
    plt.figure(figsize=(8, 6))
    plt.scatter(actual, preds, alpha=0.5)
    plt.xlabel('Actual u', fontsize=12)
    plt.ylabel('Predicted u', fontsize=12)
    plt.title(title, fontsize=14, fontweight='bold')
    plt.plot([actual.min(), actual.max()], [actual.min(), actual.max()], 'k--')
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()


def plot_solution_heatmaps(model: nn.Module, X: torch.Tensor, y: torch.Tensor,
                          title_prefix: str = ""):
    """Plot predicted, true, and error heatmaps."""
    model.eval()
    with torch.no_grad():
        u_pred = model(X).cpu().numpy()
    
    # Extract coordinates
    x_vals = X[:, 0].cpu().numpy()
    t_vals = X[:, 1].cpu().numpy()
    x_unique = np.unique(x_vals)
    t_unique = np.unique(t_vals)
    
    # Create grids
    X_grid, T_grid = np.meshgrid(x_unique, t_unique)
    U_pred_grid = u_pred.reshape(len(t_unique), len(x_unique))
    U_true_grid = y.cpu().numpy().reshape(len(t_unique), len(x_unique))
    
    fig, axes = plt.subplots(1, 3, figsize=(18, 5))
    
    # Predicted solution
    im1 = axes[0].contourf(X_grid, T_grid, U_pred_grid, levels=50, cmap='viridis')
    axes[0].set_title(f'{title_prefix} Predicted Solution u(x,t)', 
                      fontsize=14, fontweight='bold')
    axes[0].set_xlabel('x', fontsize=12)
    axes[0].set_ylabel('t', fontsize=12)
    plt.colorbar(im1, ax=axes[0], label='u')
    
    # True solution
    im2 = axes[1].contourf(X_grid, T_grid, U_true_grid, levels=50, cmap='viridis')
    axes[1].set_title('True Solution u(x,t)', fontsize=14, fontweight='bold')
    axes[1].set_xlabel('x', fontsize=12)
    axes[1].set_ylabel('t', fontsize=12)
    plt.colorbar(im2, ax=axes[1], label='u')
    
    # Absolute error
    error = np.abs(U_pred_grid - U_true_grid)
    im3 = axes[2].contourf(X_grid, T_grid, error, levels=50, cmap='hot')
    axes[2].set_title(f'{title_prefix} Absolute Error', 
                      fontsize=14, fontweight='bold')
    axes[2].set_xlabel('x', fontsize=12)
    axes[2].set_ylabel('t', fontsize=12)
    plt.colorbar(im3, ax=axes[2], label='Error')
    
    plt.tight_layout()
    plt.show()


def plot_residual_heatmap(model: nn.Module, X_coll: torch.Tensor, nu: float,
                         title_prefix: str = ""):
    """Plot physics residual heatmap."""
    X_coll_eval = X_coll.clone().detach().requires_grad_(True)
    
    model.eval()
    u_coll = model(X_coll_eval)
    
    # Compute gradients
    u_x = torch.autograd.grad(u_coll.sum(), X_coll_eval, create_graph=True)[0][:, 0:1]
    u_t = torch.autograd.grad(u_coll.sum(), X_coll_eval, create_graph=True)[0][:, 1:2]
    u_xx = torch.autograd.grad(u_x.sum(), X_coll_eval, create_graph=True)[0][:, 0:1]
    
    # Residual
    residual = u_t + u_coll * u_x - nu * u_xx
    
    # Extract coordinates
    x_coll = X_coll[:, 0].cpu().numpy()
    t_coll = X_coll[:, 1].cpu().numpy()
    residual_np = residual.detach().cpu().numpy()
    
    # Interpolate for smooth heatmap
    X_grid, T_grid = np.meshgrid(
        np.linspace(x_coll.min(), x_coll.max(), 100),
        np.linspace(t_coll.min(), t_coll.max(), 100)
    )
    residual_grid = griddata(
        (x_coll, t_coll), 
        residual_np.flatten(), 
        (X_grid, T_grid), 
        method='cubic'
    )
    
    plt.figure(figsize=(10, 6))
    im = plt.contourf(X_grid, T_grid, np.abs(residual_grid), levels=50, cmap='hot')
    plt.colorbar(im, label='|Residual|')
    plt.title(f'{title_prefix} Physics Residual Heatmap', 
              fontsize=14, fontweight='bold')
    plt.xlabel('x', fontsize=12)
    plt.ylabel('t', fontsize=12)
    plt.tight_layout()
    plt.show()
    
    print(f"Max absolute residual: {np.abs(residual_np).max():.6e}")
    print(f"Mean absolute residual: {np.abs(residual_np).mean():.6e}")


def plot_loss_curves(loss_dict: Dict[str, List[float]], title: str = "Training Loss"):
    """Plot training loss curves."""
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # Full training history
    axes[0].plot(loss_dict['loss'], label='Total Loss', linewidth=2)
    if 'data_loss' in loss_dict:
        axes[0].plot(loss_dict['data_loss'], label='Data Loss', 
                    linewidth=2, alpha=0.7)
        axes[0].plot(loss_dict['physics_loss'], label='Physics Loss', 
                    linewidth=2, alpha=0.7)
    axes[0].set_xlabel('Epoch', fontsize=12)
    axes[0].set_ylabel('Loss', fontsize=12)
    axes[0].set_title(f'{title} (Log Scale)', fontsize=14, fontweight='bold')
    axes[0].legend()
    axes[0].grid(True, alpha=0.3)
    axes[0].set_yscale('log')
    
    # Last 500 epochs
    n = min(500, len(loss_dict['loss']))
    axes[1].plot(loss_dict['loss'][-n:], label='Total Loss', linewidth=2)
    axes[1].set_xlabel('Epoch', fontsize=12)
    axes[1].set_ylabel('Loss', fontsize=12)
    axes[1].set_title(f'{title} (Last {n} Epochs)', fontsize=14, fontweight='bold')
    axes[1].legend()
    axes[1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()


def plot_comparison_bar_chart(results: Dict[str, Dict[str, float]]):
    """Bar chart comparing metrics across models."""
    models = list(results.keys())
    metrics = ['RMSE', 'MAE', 'Relative_L2_Error', 'Max_Residual']
    
    fig, axes = plt.subplots(2, 2, figsize=(14, 10))
    axes = axes.flatten()
    
    for idx, metric in enumerate(metrics):
        values = [results[model][metric] for model in models]
        axes[idx].bar(models, values, color=['blue', 'orange', 'green'][:len(models)])
        axes[idx].set_ylabel(metric, fontsize=12)
        axes[idx].set_title(f'{metric} Comparison', fontsize=13, fontweight='bold')
        axes[idx].grid(True, alpha=0.3, axis='y')
        
        # Add value labels on bars
        for i, v in enumerate(values):
            axes[idx].text(i, v, f'{v:.2e}', ha='center', va='bottom', fontsize=10)
    
    plt.tight_layout()
    plt.show()

In [9]:
# ==================== MAIN PIPELINE ====================

def run_burgers_experiment(
    data_path: str,
    collocation_path: str,
    nu: float = 0.01/np.pi,
    epochs: int = 2000,
    lr: float = 1e-3,
    alpha: float = 1.0,
    beta: float = 1.0,
    save_dir: str = 'burgers_results'
) -> Dict[str, any]:
    """
    Complete experimental pipeline for Burgers equation.
    
    Returns:
        Dictionary with models, metrics, and loss histories
    """
    
    print("="*70)
    print("BURGERS EQUATION PINN-KAN EXPERIMENT")
    print("="*70)
    
    # Load data
    print("\n1. Loading data...")
    collocation_df = pd.read_csv(collocation_path)
    X_collocation = torch.tensor(collocation_df.values, dtype=torch.float32).to(device)
    
    full_df = pd.read_csv(data_path)
    X_full = torch.tensor(full_df[['x', 't']].values, dtype=torch.float32).to(device)
    y_full = torch.tensor(full_df['u'].values, dtype=torch.float32).unsqueeze(-1).to(device)
    
    print(f"   X_full shape: {X_full.shape}")
    print(f"   y_full shape: {y_full.shape}")
    print(f"   X_collocation shape: {X_collocation.shape}")
    
    # Initialize models
    print("\n2. Initializing models...")
    models = {
        'PINN-KAN': PINN_KAN(input_dim=2, num_rbfs_list=[20, 30, 15], out_dim=1).to(device),
        'Vanilla-MLP': VanillaMLP(input_dim=2, hidden_dims=[64, 64, 32], out_dim=1).to(device),
        'Vanilla-PINN': VanillaPINN(input_dim=2, hidden_dims=[64, 64, 32], out_dim=1).to(device)
    }
    
    for name, model in models.items():
        n_params = sum(p.numel() for p in model.parameters())
        print(f"   {name}: {n_params:,} parameters")
    
    # Train models
    print("\n3. Training models...")
    loss_histories = {}
    
    for name, model in models.items():
        print(f"\n{'='*70}")
        print(f"Training {name}")
        print(f"{'='*70}")
        
        if name == 'Vanilla-MLP':
            loss_dict = train_vanilla_mlp(model, X_full, y_full, epochs=epochs, lr=lr)
        else:
            loss_dict = train_pinn(model, X_full, y_full, X_collocation, nu, 
                                  epochs=epochs, lr=lr, alpha=alpha, beta=beta)
        
        loss_histories[name] = loss_dict
    
    # Evaluate models
    print("\n4. Evaluating models...")
    all_metrics = {}
    
    for name, model in models.items():
        print(f"\nEvaluating {name}...")
        metrics = compute_metrics(model, X_full, y_full, X_collocation, nu)
        all_metrics[name] = metrics
        
        print(f"   RMSE: {metrics['RMSE']:.6e}")
        print(f"   MAE: {metrics['MAE']:.6e}")
        print(f"   Max Residual: {metrics['Max_Residual']:.6e}")
    
    # Visualizations
    print("\n5. Generating visualizations...")
    
    for name, model in models.items():
        print(f"\nVisualizations for {name}:")
        
        # Prediction vs actual
        plot_prediction_vs_actual(model, X_full, y_full, title=f"{name}: Predictions vs Actual")
        
        # Solution heatmaps
        plot_solution_heatmaps(model, X_full, y_full, title_prefix=name)
        
        # Residual heatmap (only for PINN models)
        if 'PINN' in name or 'KAN' in name:
            plot_residual_heatmap(model, X_collocation, nu, title_prefix=name)
        
        # Loss curves
        plot_loss_curves(loss_histories[name], title=f"{name} Training")
    
    # Comparison plot
    print("\nGenerating comparison chart...")
    plot_comparison_bar_chart(all_metrics)
    
    # Save results
    print(f"\n6. Saving results to {save_dir}...")
    os.makedirs(save_dir, exist_ok=True)
    
    # Save models
    for name, model in models.items():
        model_path = os.path.join(save_dir, f"{name.lower().replace('-', '_')}_model.pth")
        torch.save(model.state_dict(), model_path)
    
    # Save metrics
    metrics_df = pd.DataFrame(all_metrics).T
    metrics_df.to_csv(os.path.join(save_dir, 'metrics_comparison.csv'))
    
    with open(os.path.join(save_dir, 'all_results.pkl'), 'wb') as f:
        pickle.dump({
            'metrics': all_metrics,
            'loss_histories': loss_histories
        }, f)
    
    print("\n" + "="*70)
    print("EXPERIMENT COMPLETED!")
    print("="*70)
    
    # Print summary table
    print("\nFINAL METRICS COMPARISON:")
    print(metrics_df.to_string())
    
    return {
        'models': models,
        'metrics': all_metrics,
        'loss_histories': loss_histories
    }

In [ ]:
# ==================== USAGE EXAMPLE ====================

if __name__ == "__main__":
    # Run complete experiment
    results = run_burgers_experiment(
        data_path='../data/burgers_full.csv',
        collocation_path='../data/burgers_collocation.csv',
        nu=0.01/np.pi,
        epochs=2000,
        lr=1e-3,
        alpha=1.0,
        beta=1.0,
        save_dir='../Results/burgers_results'
    )
    
    print("\n✅ All experiments completed successfully!")
    print(f"📊 Results saved in 'burgers_results/' directory")


BURGERS EQUATION PINN-KAN EXPERIMENT

1. Loading data...
   X_full shape: torch.Size([25600, 2])
   y_full shape: torch.Size([25600, 1])
   X_collocation shape: torch.Size([20000, 2])

2. Initializing models...
   PINN-KAN: 2,991 parameters
   Vanilla-MLP: 6,465 parameters
   Vanilla-PINN: 6,465 parameters

3. Training models...

Training PINN-KAN
Epoch 1/2000 | Total: 4.973315e-01 | Data: 4.973315e-01 | Physics: 1.800404e-10
